# Feature Engineering

- 1. [Introduction](#1.-Introduction)
- 2. [Imports](#2.-Imports)
- 3. [Loading the data](#3.-Loading-the-data)
- 4. [Partitioning Our Dataset](#4.-Partitioning-Our-Dataset)
- 5. [Defining Our Preprocessing Pipeline](#5.-Defining-Our-Preprocessing-Pipeline)
    - 5.1 Pipeline Definition
    - 5.2 Fitting and Transforming Data
    - 5.4 Fitting and Transforming Data for Computing LACE Index
- 6. [Exporting Preprocessed Data](#6.-Exporting-Preprocessed-Data)

## 1. Introduction
This notebook is dedicated to developing features for our later models to ingest. It includes imputation, scaling, one-hot encoding, transformation, outlier handling, and data splitting. Methods for re-sampling the dataset to handle class imbalances will not occur in this phase, but rather the model evaluation phase.

## 2. Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns

from category_encoders import OrdinalEncoder

from sklearn.feature_selection import VarianceThreshold 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearndf.transformation import SimpleImputerDF, StandardScalerDF, FunctionTransformerDF, VarianceThresholdDF, OneHotEncoderDF

from src.data import load_dataset as ld
from src.features.icd9 import icd9_to_classification, icd9_to_category

from src.features.transformer import PandasFeatureUnion, RowFilter, ColumnSelector, ColumnFilter, \
DiagnosisMapper, EncodeNaNCategoricalImputer, MostFrequentCategoricalImputer, CategoryCollapseThreshold, \
CategoricalHomogeneityThreshold, FeatureCombiner, DiagnosisComorbidityExtractor, CollinearityThreshold, OneHotEncoder

sns.set()
pd.options.display.max_columns = 100

RANDOM_SEED = 0

## 3. Loading the data

In [3]:
df = ld.load_interim_pickle('00_diabetes.pkl')
df.head()

,,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,days_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,is_readmitted_early
encounter_id,patient_nbr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12522,48330783,Caucasian,Female,[80-90),2,1,4,13,NA,Not Available,68,2,28,0,0,0,398,427,38,8,Not Available,Not Available,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0
15738,63555939,Caucasian,Female,[90-100),3,3,4,12,NA,InternalMedicine,33,3,18,0,0,0,434,198,486,8,Not Available,Not Available,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0
16680,42519267,Caucasian,Male,[40-50),1,1,7,1,NA,Not Available,51,0,8,0,0,0,197,157,250,5,Not Available,Not Available,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0
28236,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,NA,Not Available,47,2,17,0,0,0,250.7,403,996,9,Not Available,Not Available,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0
35754,82637451,Caucasian,Male,[50-60),2,1,2,3,NA,Not Available,31,6,16,0,0,0,414,411,250,9,Not Available,Not Available,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0


## 4. Partitioning Our Dataset
We must first partition our training and test sets. The two sets should be stratified to ensure that we have an approximately equal distribution of positive and negative observations. We will be fitting our preprocessor on our training set and preprocessing both the training and test set.

In [4]:
X = df.drop(columns=['is_readmitted_early'])
y = df.is_readmitted_early

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((57213, 46), (14304, 46), (57213,), (14304,))

In [5]:
y_train.value_counts(normalize=True)

0    0.912013
1    0.087987
Name: is_readmitted_early, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

0    0.911983
1    0.088017
Name: is_readmitted_early, dtype: float64

## 5. Defining Our Preprocessing Pipeline
We will be building two preprocessors: one that includes one-hot encoding and one that doesn't. Tree-based algorithms don't require that categorical variables be encoded numerically, so we will use the former to speed up training for those kinds of models.

In [7]:
# Columns that we found were homogeneous or had a high NaN values in EDA
columns_to_drop = [
    'payer_code',
    'examide',
    'citoglipton',
    'glimepiride-pioglitazone'
]

categorical_features = X.select_dtypes(exclude=[np.number]).drop(columns=columns_to_drop)
nominal_features = categorical_features.drop(columns=['age']).columns.tolist()
ordinal_features = ['age']
numerical_features = X.select_dtypes(include=[np.number]).columns.tolist()
target_feature = ['is_readmitted_early']


# Features where we want null/nan values to be encoded as their own category
encode_nan_as_category_features = ['diag_1', 'diag_2', 'diag_3', 'gender', 'max_glu_serum', 'medical_specialty', 'A1Cresult']

# We created custom categories for the following columns to indicate unavailability
encode_nan_as_category_special_cases = {
    'admission_type_id': 5,
    'discharge_disposition_id': 25,
    'admission_source_id': 15,
    'race': 'Unknown/Invalid'
}

# For the remaining nominal features, we will just use the most frequent category in our training set
most_frequent_category_features = list(set(nominal_features) - set(encode_nan_as_category_features))

# When merging small categories into one category, we have label specifications for the following columns
merge_categories_special_cases = {
    'discharge_disposition_id': 30,
    'admission_type_id': 9,
    'admission_source_id': 27  
}

row_filters = {
    # Expired or Hospice-related discharges are not likely to be readmitted. Remove neonatal observations
    'discharge_disposition_id': lambda s: ~s.isin([11, 13, 14, 19, 20, 21, 10]),
    # We remove observations related to birth or infancy
    'admission_source_id': lambda s: ~s.isin([11, 12, 13, 14])
}
 
def compute_entropy(d1, d2, d3):
    diagnoses = [d1, d2, d3]
    num_diagnoses = len(diagnoses)
    prob_d1 = diagnoses.count(d1) / num_diagnoses
    prob_d2 = diagnoses.count(d2) / num_diagnoses
    prob_d3 = diagnoses.count(d3) / num_diagnoses
    return -(prob_d1 * np.log(prob_d1) + prob_d2 * np.log(prob_d2) + prob_d3 * np.log(prob_d3)) 

def diagnosis_diversity(r):
    d1 = icd9_to_category(r.diag_1)
    d2 = icd9_to_category(r.diag_2)
    d3 = icd9_to_category(r.diag_3)
    return compute_entropy(d1, d2, d3)

combined_numerical_features = [
    (('number_inpatient', 'number_outpatient', 'number_emergency'), np.sum, 'service_utilization', int)
]

combined_nominal_features = [
    (('diag_1', 'diag_2', 'diag_3'), diagnosis_diversity, 'diagnosis_diversity', float)
]


### 5.1 Pipeline Definition
1. **ColumnFilter**: filter out unwanted columns based on our prior exploratory data analysis. These columns are defined in the `columns_to_drop` variable
2. **RowFilter**: drop observations that have invalid values. These are defined in `row_filters`.
3. **Numerical Features Sub-pipeline:**
    - **ColumnFilter**: only select numerical features (e.g. `numerical_features`)
    - **SimpleImputer**: Impute missing values with the median of each column.
    - **Log1pTransformer**: Log-transform all columns for additivity and to promote normality.
    - **StandardScaler**: Centering the data (mean=0, std=1).
    - **FeatureCombiner**: Create compound features composed of various existing features. These are defined in `combined_numerical_features`.
    - **VarianceThreshold**: Remove homogeneous features that have a variance lower than a particular threshold. I chose 0.1 for this project.
4. **Ordinal Features Sub-pipeline:**
    - **ColumnFilter**: only select ordinal features (e.g. `ordinal_features`)
    - **SimpleImputer**: Impute missing values with the most frequently occurring feature value.
    - **OrdinalEncoder**: Map each category to an index.
    
5. **Nominal Features Sub-pipeline:**
    - **ColumnFilter**: only select nominal features (e.g. `nominal_features`)
    - **EncodeNaNCategoricalImputer**: For some features (`encode_nan_as_category_features`), we want to encode NaN as its own category because it contains some useful information.
    - **MostFrequentImputer**: Impute missing values with the most frequently occurring feature value. This custom imputer only does this for a subset of features (`most_frequent_category_features`).
    - **FeatureCombiner**:  Create compound features composed of various existing features. These are defined in `combined_nominal_features`. We mainly use this to create `diagnosis_diversity`, which measures the heterogeneity in the primary, secondary, and tertiary diagnosis reported.
    - **DiagnosisMapper**: Converts ICD9 codes into two tiers of classifications (t1, t2), differing by specificity. t2 is more general than t1.
    - **HomogeneityThreshold**: Remove features if they're too homogeneous, based on Shannon Entropy. I set this threshold to be below 0.05.
    - **CategoryCollapseThreshold**: Collapse feature values that occur less than X% of the time into a single value. I set this threshold to be below 5%.
6. **CollinearityThreshold**: Remove one of each pair of correlated features where their absolute Pearson correlation coefficient is greater than 0.9.

In [8]:
def create_pipeline():
    nominal_transformers = [
        ('column_selector', ColumnSelector(nominal_features)),
        ('encode_nan_as_category_imputer', EncodeNaNCategoricalImputer(encode_nan_as_category_features, special_cases=encode_nan_as_category_special_cases)),
        ('most_frequent_imputer', MostFrequentCategoricalImputer(most_frequent_category_features)),
        ('feature_combiner', FeatureCombiner(combined_nominal_features)),
        ('comorbidity_extractor', DiagnosisComorbidityExtractor()),
        ('diagnosis_mapper', DiagnosisMapper()),
        ('homogeneity_threshold', CategoricalHomogeneityThreshold(threshold=0.05, verbose=True)),
        ('category_collapse_threshold', CategoryCollapseThreshold(threshold=0.05, special_cases=merge_categories_special_cases, verbose=True)),
        ('one_hot_encoder', OneHotEncoder(drop_first=True, columns_to_ignore=['diagnosis_diversity']))
    ]
    
    return Pipeline([
        ('column_filter', ColumnFilter(columns_to_drop)),
        ('row_filter', RowFilter(row_filters)),
        ('features', PandasFeatureUnion([
            ('numerical', Pipeline([
                ('column_selector', ColumnSelector(numerical_features)),
                ('simple_imputer', SimpleImputerDF(strategy='median')),
                ('log1p_transformer', FunctionTransformerDF(np.log1p)),
                ('standard_scaler', StandardScalerDF()),
                ('feature_combiner', FeatureCombiner(combined_numerical_features)),
                ('variance_threshold', VarianceThresholdDF(0.1))
            ])),
            ('ordinal', Pipeline([
                ('column_selector', ColumnSelector(ordinal_features)),
                ('simple_imputer', SimpleImputerDF(strategy='most_frequent')),
                ('ordinal_encoder', OrdinalEncoder(return_df=True))
            ])),
            ('nominal', Pipeline(nominal_transformers)),
        ])),
        ('collinearity_threshold', CollinearityThreshold(verbose=True))
    ])


preprocessor = create_pipeline()

In order to compute the LACE index as our benchmark model, we'll have to generate a dataset with only a subset of untransformed features:

In [9]:
def create_pipeline_for_lace_index():
    columns = [
        'admission_type_id',
        'diag_1',
        'diag_2',
        'diag_3',
        'number_emergency',
        'days_in_hospital',
    ]
    
    nominal_transformers = [
        ('column_selector', ColumnSelector(['admission_type_id', 'diag_1', 'diag_2', 'diag_3'])),
        ('encode_nan_as_category_imputer', EncodeNaNCategoricalImputer(['diag_1', 'diag_2', 'diag_3'], special_cases={'admission_type_id': 5})),
        ('comorbidity_extractor', DiagnosisComorbidityExtractor())
    ]
    
    return Pipeline([
        ('row_filter', RowFilter(row_filters)),
        ('column_selector', ColumnSelector(columns)),
        ('features', PandasFeatureUnion([
            ('numerical', Pipeline([
                ('column_selector', ColumnSelector(['number_emergency', 'days_in_hospital'])),
                ('simple_imputer', SimpleImputerDF(strategy='median'))
            ])),
            ('nominal', Pipeline(nominal_transformers))
        ])),
        ('column_filter', ColumnFilter(['diag_1', 'diag_2', 'diag_3']))
    ])


lace_index_preprocessor = create_pipeline_for_lace_index()

### 5.2 Fitting and Transforming Data

In [10]:
_ = preprocessor.fit(X_train, y_train)

[CategoricalHomogeneityThreshold] Column: nateglinide, Entropy: 0.04319967603513349. Dropping column
[CategoricalHomogeneityThreshold] Column: chlorpropamide, Entropy: 0.008258518423868332. Dropping column
[CategoricalHomogeneityThreshold] Column: acetohexamide, Entropy: -0.0. Dropping column
[CategoricalHomogeneityThreshold] Column: tolbutamide, Entropy: 0.002325111500385499. Dropping column
[CategoricalHomogeneityThreshold] Column: acarbose, Entropy: 0.020170795097038747. Dropping column
[CategoricalHomogeneityThreshold] Column: miglitol, Entropy: 0.002602731822575121. Dropping column
[CategoricalHomogeneityThreshold] Column: troglitazone, Entropy: 0.00021324454948446096. Dropping column
[CategoricalHomogeneityThreshold] Column: tolazamide, Entropy: 0.0036157389239628275. Dropping column
[CategoricalHomogeneityThreshold] Column: glyburide-metformin, Entropy: 0.04329416024461899. Dropping column
[CategoricalHomogeneityThreshold] Column: glipizide-metformin, Entropy: 0.0010873373216703

[CollinearityThreshold] Correlated features: [('diag_3_t1_Genitourinary', 'diag_3_t2_Genitourinary'), ('diag_3_t1_Other', 'diag_3_t2_Other'), ('diag_3_t1_Respiratory', 'diag_3_t2_Respiratory'), ('diag_2_t1_Genitourinary', 'diag_2_t2_Genitourinary'), ('diag_2_t1_Other', 'diag_2_t2_Other'), ('diag_2_t1_Respiratory', 'diag_2_t2_Respiratory'), ('diag_1_t1_Digestive', 'diag_1_t2_Digestive'), ('diag_1_t1_Injury/Poison', 'diag_1_t2_Injury'), ('diag_1_t1_Musculoskeletal', 'diag_1_t2_Musculoskeletal'), ('diag_1_t1_Other', 'diag_1_t2_Other'), ('diag_1_t1_Respiratory', 'diag_1_t2_Respiratory')]


In [11]:
X_train_enc = preprocessor.transform(X_train)

In [12]:
X_test_enc = preprocessor.transform(X_test)

In [13]:
train_enc = pd.concat([X_train_enc, y_train[X_train_enc.index]], axis=1)
test_enc = pd.concat([X_test_enc, y_test[X_test_enc.index]], axis=1)
train_enc.shape, test_enc.shape

((55956, 81), (14007, 81))

### 5.3 Fitting and Transforming Data for Computing LACE Index

In [14]:
X_train_lace = lace_index_preprocessor.fit_transform(X_train)
train_lace = pd.concat([X_train_lace, y_train[X_train_lace.index]], axis=1)
train_lace.shape

(55956, 21)

In [15]:
X_test_lace = lace_index_preprocessor.fit_transform(X_test)
test_lace = pd.concat([X_test_lace, y_test[X_test_lace.index]], axis=1)
test_lace.shape

(14007, 21)

## 6. Exporting Preprocessed Data

In [16]:
train_enc.to_pickle(ld.find_preprocessed_path('train.pkl'))
test_enc.to_pickle(ld.find_preprocessed_path('test.pkl'))
train_lace.to_pickle(ld.find_preprocessed_path('train_lace.pkl'))
test_lace.to_pickle(ld.find_preprocessed_path('test_lace.pkl'))